Install library

In [ ]:
!pip install openpyxl pandas

Import library

In [ ]:
import pandas as pd
import openpyxl

Import Dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/capstone/shuffled_data.csv')
data.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,product_name,label
0,Pantene Conditioner Tanpa Bilas Perfect On 40Ml,kecantikan
1,Implora Cheek & Lip Tint 04 Red Wine 5.5g,kecantikan
2,Prime Bread Roti Manis Isi Krim Keju,product-segar
3,Garnier Hair Color Naturals Express 1 Hitam,kecantikan
4,So Klin Pembersih Lantai Sereh & Geranium 770mL,home-living


In [ ]:
from openpyxl import Workbook

Function for generate date and time randomly

In [ ]:
import random
from datetime import datetime, timedelta

def generate_random_datetime():
    start_date = datetime(2020, 1, 1)
    end_date = datetime(2024, 11, 27)
    random_date = start_date + timedelta(
        seconds=random.randint(0, int((end_date - start_date).total_seconds()))
    )
    return random_date.strftime("%d.%m.%y-%H:%M")


Function for generate product names randomly from dataset

In [ ]:
def generate_random_products():
  products = {}
  number_of_products = random.randint(1,16)
  sampled_titles = data['product_name'].sample(number_of_products).str.upper()

  for title in sampled_titles:
    random_price = random.randint(10, 900) * 100
    products[title] = random_price

  products = manipulated_products_name(products)

  return products

Manipulate the name of product because some store has different way to named its items. Sometimes, the store simplify the name of product by deleting vocal, some information, etc without reduce the main information.

In [ ]:
def manipulated_products_name(products):
    max_length = 20
    manipulated_products = {}

    for product, price in products.items():
        product_name = product.split()

        # Seperate in to parts
        brand = ' '.join(product_name[:2])  # product name
        size = product_name[-1]  # size
        middle_words = product_name[2:-1]  # description name of products

        manipulated_name = f"{brand} {size}".strip()

        for word in middle_words:
            if len(manipulated_name) + len(word) + 1 <= max_length:
                manipulated_name = f"{brand} {word} {size}".strip()
            else:
                break

       # delete vocal letter if still too large
        if len(manipulated_name) > max_length:
            manipulated_name = ' '.join(
                ''.join([ch for ch in word if ch.upper() not in ['A', 'I', 'U', 'E', 'O']])
                if word not in [brand, size] else word
                for word in manipulated_name.split()
            )

        # cut another words if still larger than max length
        if len(manipulated_name) > max_length:
            manipulated_name = manipulated_name[:max_length]

        # store the manipulated product with its price
        manipulated_products[manipulated_name] = price

    return manipulated_products

Function for create border of the cell

In [ ]:
from openpyxl.styles import Border, Side

def apply_outside_border(ws):
    # Define border styles
    thin_border = Side(border_style="thin", color="000000")

    # Get the dimensions of the worksheet
    min_row, max_row = 1, ws.max_row
    min_col, max_col = 1, ws.max_column

    # Apply border to the first and last row
    for col in range(min_col, max_col + 1):
        # Top row
        cell = ws.cell(row=min_row, column=col)
        cell.border = Border(
            top=thin_border,
            left=cell.border.left,
            right=cell.border.right,
            bottom=cell.border.bottom,
        )
        # Bottom row
        cell = ws.cell(row=max_row, column=col)
        cell.border = Border(
            bottom=thin_border,
            left=cell.border.left,
            right=cell.border.right,
            top=cell.border.top,
        )

    # Apply border to the first and last column
    for row in range(min_row, max_row + 1):
        # Left column
        cell = ws.cell(row=row, column=min_col)
        cell.border = Border(
            left=thin_border,
            top=cell.border.top,
            bottom=cell.border.bottom,
            right=cell.border.right,
        )
        # Right column
        cell = ws.cell(row=row, column=max_col)
        cell.border = Border(
            right=thin_border,
            top=cell.border.top,
            bottom=cell.border.bottom,
            left=cell.border.left,
        )


Function for set the print area

In [ ]:
def apply_print_area(ws):
    min_row, max_row = 1, ws.max_row
    ws.print_area = f"A{min_row}:D{max_row}"


function for change the entire fonts

In [ ]:
def change_font(ws):
    font_calibri = Font(name="Calibri", size=9, bold=False, italic=False)
    font_consolas = Font(name="Consolas", size=9, bold=False, italic=False)

    for row in ws.iter_rows():
        if row[0].row < 6:
            for cell in row:
                cell.font = font_calibri
        else:
            for cell in row:
                cell.font = font_consolas

Function for create the worksheet

In [ ]:
from openpyxl.styles import Alignment, Border, Side, Font
from openpyxl.utils import get_column_letter

def create_receipts(worksheet_name, wb):
    # Create a new worksheet or get the existing one
    if worksheet_name in wb.sheetnames:
        ws = wb[worksheet_name]
    else:
        ws = wb.create_sheet(title=worksheet_name)

    # nambahin kop
    ws['A1'] = "PT. INDOMARET PRISMATAMA"
    ws['A2'] = "JL ANCOL U9, ANCOL"
    ws['A3'] = "BARAT- JAKARTA UTARA"
    ws['A4'] = "NPWP 123456789"

    # alamat indomaret
    ws['A6'] = "KS TUBUN 0293493733"
    ws['A7'] = "JL KS TUBUN NO.05 RTO2 RWO4 KEL TEMANGGUNG"
    ws['A8'] = "KEC TEMANGGUNG KAB TEMANGGUNG, 56213"
    for i in range(3):
      ws.merge_cells(start_row=6+i, start_column=1, end_row=6+i, end_column=4)
      ws['A'+str(6+i)].alignment = Alignment(horizontal='center')


    # informasi bon
    ws['A9'] = generate_random_datetime() + "/2.1.27/FZIJ"
    ws.merge_cells(start_row=9, start_column=1, end_row=9, end_column=2)
    ws['C9'].alignment = Alignment(horizontal='left')
    ws['C9'] = "301135/RATIH/01"
    ws.merge_cells(start_row=9, start_column=3, end_row=9, end_column=4)
    ws['C9'].alignment = Alignment(horizontal='right')
    # border informasi bon
    dashed_border = Border(
    top=Side(border_style="dashed", color="000000"),
    bottom=Side(border_style="dashed", color="000000")
    )
    # Terapkan border pada seluruh sel di baris 9
    for col in range(1, 5):  # Kolom A hingga D
      cell = ws.cell(row=9, column=col)
      cell.border = dashed_border

    # informasi product
    products = generate_random_products()
    for i, (product_name, product_price) in enumerate(products.items()):
      random_amount_item = random.randint(1, 8)
      ws.cell(row=10 + i, column=1, value=product_name)
      ws.cell(row=10 + i, column=2, value=random_amount_item)
      ws.cell(row=10 + i, column=3, value=product_price)
      ws.cell(row=10 + i, column=4, value=product_price * random_amount_item)

    max_length = 20
    ws.column_dimensions['B'].width = 3
    ws.column_dimensions['C'].width = 9
    ws.column_dimensions['D'].width = 9
    ws.column_dimensions[get_column_letter(1)].width = max_length

    # informasi uang
    start_row_info = 10 + len(products)
    total_cost = sum(product_price * random_amount_item for product_price in products.values())
    random_cash = total_cost + (random.randint(0, 100) * 1000)

    # Isi data dan lakukan merge
    # Membuat border dengan dashed line
    dashed_border_top = Border(
      top=Side(border_style="dashed", color="000000"))

    # Menambahkan label dan nilai untuk "HARGA JUAL", "TUNAI", "KEMBALI"
    for i, (label, value) in enumerate([
      ("HARGA JUAL", total_cost),
      ("TUNAI", random_cash),
      ("KEMBALI", random_cash - total_cost)
    ]):
    # Menambahkan label di kolom B
      ws['B' + str(start_row_info + i)] = label
      ws['B' + str(start_row_info + i)].alignment = Alignment(horizontal='right')

      # Menambahkan nilai di kolom D
      ws['D' + str(start_row_info + i)] = value

      # Merge kolom B dan C
      ws.merge_cells(start_row=start_row_info + i, start_column=2, end_row=start_row_info + i, end_column=3)

      # Jika ini adalah label "HARGA JUAL", tambahkan border dashed di atas
      if label == "HARGA JUAL":
        # Terapkan border dashed pada baris yang berisi "HARGA JUAL"
        for col in range(2, 5):  # Kolom B until D
            cell = ws.cell(row=start_row_info + i, column=col)
            cell.border = dashed_border_top


    # footer receipt
    start_row_footer = start_row_info + 3
    ws['A' + str(start_row_footer)] = "PPN : DPP = 8,919 PPN = 981"
    ws['A' + str(start_row_footer+1)] = "NON PPN : DPP = 100"
    ws['A' + str(start_row_footer+2)] = "LAYANAN KONSUMEN"
    ws['A' + str(start_row_footer+3)] = "SMS/WA 0811.1500.280 TELP 1500280"
    ws['A' + str(start_row_footer+4)] = "KONTAK@INDOMARET.CO.ID"
    ws['A' + str(start_row_footer+5)] = "BELANJA LEBIH MUDAH DI KLIKINDOMARET"
    ws['A' + str(start_row_footer+6)] = "GRATIS ONGKIR 1 JAM SAMPAI"
    for i in range(7):
      ws.merge_cells(start_row=start_row_footer+i, start_column=1, end_row=start_row_footer+i, end_column=4)
      if i > 1:
        ws['A'+str(start_row_footer+i)].alignment = Alignment(horizontal='center')

    # Set the border
    apply_outside_border(ws)

    # Set the print area
    apply_print_area(ws)

    # change the fonts
    change_font(ws)



    return ws


In [ ]:
# Create a new workbook
wb = Workbook()

# Dictionary to store worksheets
worksheets = {}

for i in range(10):
    sheet_name = f"sheet{i+1}"
    worksheets[sheet_name] = wb.create_sheet(title=sheet_name)
    create_receipts(sheet_name, wb)

# Save the workbook to a file
wb.save("nota.xlsx")
# Print a success message
print("Excel file created successfully!")

Excel file created successfully!
